In [8]:
import os
import pprint

# Read a file and split into lines
def read_norm_lines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [line.split('||') for line in lines]

def make_dictionary_annotations_and_text(name):
    text_file_path = str(name) + ".txt"
    text_norm_path = str(name) + ".norm"
    note_text = open(os.path.join('/Users/isabelmetzger/PycharmProjects/ClinicalScorecard/data/train/train_note', text_file_path)).read()
    list_norm_annotations = read_norm_lines(os.path.join('/Users/isabelmetzger/PycharmProjects/ClinicalScorecard/data/train/train_norm', text_norm_path))
    annotation_dictionary = {'id': [], 'concept': [], 'beginCharacterOffset': [],
                             'endCharacterOffset': [], 'text': [], 'additional': {'beginCharacterOffset': [],
                                                                                  'endCharacterOffset': [],
                                                                                  'text': []}}
    for list_norm in list_norm_annotations:
        annotation_dictionary['id'].append(list_norm[0])
        annotation_dictionary['concept'].append(list_norm[1])
        annotation_dictionary['beginCharacterOffset'].append(list_norm[2])
        annotation_dictionary['endCharacterOffset'].append(list_norm[3])
        annotated_text = note_text[int(list_norm[2]): int(list_norm[3])]
        annotation_dictionary['text'].append(annotated_text)
        if len(list_norm) > 5:
            annotation_dictionary['additional']['beginCharacterOffset'].append(list_norm[4])
            annotation_dictionary['additional']['endCharacterOffset'].append(list_norm[5])
            annotation_dictionary['additional']['text'].append(note_text[int(list_norm[4]): int(list_norm[5])])
            
    return annotation_dictionary #.items())# list_norm_annotations


pprint.pprint(make_dictionary_annotations_and_text('0038'))


AttributeError: 'dict' object has no attribute 'to_dict'

In [1]:
import glob
import unicodedata

def findFiles(path):
    """
    returns a list of files in a path
    """
    return glob.glob(path)


def unicodeToAscii(s):
    """
    Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
    """
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )


def normalizeStringAndDigitsLower(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([,.:!?])", r" \1 ", s)
    s = re.sub(r"[\|]+", r" ", s)
    s = re.sub('\n', '<NEWLINE>', s)
    s = re.sub('\t', ' ', s)
    s = re.sub('\d+', '#', s)
    s = re.sub(' +', ' ',s)
    return s


def build_vocab(vocab_min, infile, vocab_filename):
    """
        INPUTS:
            vocab_min: how many documents a word must appear in to be kept
            infile: (training) data file to build vocabulary from
            vocab_filename: name for the file to output
    """
    with open(infile, 'r') as csvfile:
        reader = csv.reader(csvfile)
        #header
        next(reader)

        #0. read in data
        print("reading in data...")
        #holds number of terms in each document
        note_numwords = []
        #indices where notes start
        note_inds = [0]
        #indices of discovered words
        indices = []
        #holds a bunch of ones
        data = []
        #keep track of discovered words
        vocab = {}
        #build lookup table for terms
        num2term = {}
        #preallocate array to hold number of notes each term appears in
        note_occur = np.zeros(400000, dtype=int)
        i = 0
        for row in reader:
            text = row[2]
            numwords = 0
            for term in text.split():
                #put term in vocab if it's not there. else, get the index
                index = vocab.setdefault(term, len(vocab))
                indices.append(index)
                num2term[index] = term
                data.append(1)
                numwords += 1
            #record where the next note starts
            note_inds.append(len(indices))
            indset = set(indices[note_inds[-2]:note_inds[-1]])
            #go thru all the word indices you just added, and add to the note occurrence count for each of them
            for ind in indset:
                note_occur[ind] += 1
            note_numwords.append(numwords)
            i += 1
        #clip trailing zeros
        note_occur = note_occur[note_occur>0]

        #turn vocab into a list so indexing doesn't get fd up when we drop rows
        vocab_list = np.array([word for word,ind in sorted(vocab.items(), key=operator.itemgetter(1))])

        #1. create sparse document matrix
        C = csr_matrix((data, indices, note_inds), dtype=int).transpose()
        #also need the numwords array to be a sparse matrix
        note_numwords = csr_matrix(1. / np.array(note_numwords))
        
        #2. remove rows with less than 3 total occurrences
        print("removing rare terms")
        #inds holds indices of rows corresponding to terms that occur in < 3 documents
        inds = np.nonzero(note_occur >= vocab_min)[0]
        print(str(len(inds)) + " terms qualify out of " + str(C.shape[0]) + " total")
        #drop those rows
        C = C[inds,:]
        note_occur = note_occur[inds]
        vocab_list = vocab_list[inds]

        print("writing output")
        with open(vocab_filename, 'w') as vocab_file:
            for word in vocab_list:
                vocab_file.write(word + "\n")


def letterToIndex(letter):
    """
    find index for all letters
    """
    return all_letters.find(letter)


def letterToTensor(letter):
    """
    Just for demonstration, turn a letter into a <1 x n_letters> Tensor
    """
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor


def lineToTensor(line):
    """# Turn a line into a <line_length x 1 x n_letters>, or an array of one-hot letter vectors"""
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


all_CNER_words = {}
all_CNER_labels = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

